In [1]:
import sys
import os

%load_ext autoreload
%autoreload 2

In [2]:
import zarr
import numpy as np
import torch
import copy
import numcodecs

from pathlib import Path

import gym_pusht  # noqa: F401
import gymnasium as gym
import imageio
import numpy
import torch
from huggingface_hub import snapshot_download

from lerobot.common.policies.diffusion.modeling_diffusion import DiffusionPolicy

from lerobot.common.datasets.push_dataset_to_hub.utils import (
    concatenate_episodes,
    get_default_encoding,
    save_images_concurrently,
)
from lerobot.common.datasets.utils import (
    calculate_episode_data_index,
    hf_transform_to_torch,
)
from lerobot.common.datasets.lerobot_dataset import CODEBASE_VERSION
from IPython.display import Video
from lerobot.common.datasets.push_dataset_to_hub.pusht_zarr_format import to_hf_dataset, calculate_episode_data_index

/home/sm/anaconda3/envs/lerobot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
ep_dict = {
    'observation.image': [
        {'path': 'videos/observation.image_episode_000000.mp4', 'timestamp': 0.0},
        {'path': 'videos/observation.image_episode_000000.mp4', 'timestamp': 0.1}],
     'observation.state': torch.tensor([[152.0000, 398.0000], [157.1967, 399.2942]]).numpy(),
     'action': torch.tensor([[169.5758, 402.3773], [190.2249, 404.4283]]).numpy(),
     'episode_index': [0, 0],
     'frame_index': [0, 1],
     'timestamp': [0.0, 0.1],
     'next.reward': [0., 1.],
     'next.done': [False, True],
     'next.success': [False, True]
}

In [4]:
output_path = "outputs/eval/dataset_test/zarr_test"

root = zarr.open(output_path, mode='a')

In [5]:
def append_episode_to_zarr(ep_dict, zarr_group):
    for key, value in ep_dict.items():
        # 첫 번째 에피소드인 경우 해당 key로 데이터셋 생성
        if key not in zarr_group:
            data_shape = (0, *np.array(value).shape[1:])
            
            # object_codec 추가: 문자열 데이터를 저장하기 위해 JSON을 사용
            object_codec = numcodecs.JSON() if isinstance(value[0], dict) or isinstance(value[0], str) else None
            zarr_group.create_dataset(
                key,
                shape=data_shape,
                chunks=(1, *data_shape[1:]),
                dtype=np.array(value).dtype,
                maxshape=(None, *data_shape[1:]),
                object_codec=object_codec
            )
        
        # 데이터셋에 에피소드 데이터를 추가
        zarr_group[key].append(value)

In [6]:
for i in range(6):
    new_ep_dict = copy.deepcopy(ep_dict)
    new_ep_dict['episode_index'] = [i] * 2

    append_episode_to_zarr(new_ep_dict, root)

In [7]:
zarr_data = zarr.open(output_path, mode='r')

In [8]:
list(zarr_data['observation.image'])

[{'path': 'videos/observation.image_episode_000000.mp4', 'timestamp': 0.0},
 {'path': 'videos/observation.image_episode_000000.mp4', 'timestamp': 0.1},
 {'path': 'videos/observation.image_episode_000000.mp4', 'timestamp': 0.2},
 {'path': 'videos/observation.image_episode_000000.mp4', 'timestamp': 0.3},
 {'path': 'videos/observation.image_episode_000000.mp4', 'timestamp': 0.4},
 {'path': 'videos/observation.image_episode_000000.mp4', 'timestamp': 0.5},
 {'path': 'videos/observation.image_episode_000000.mp4', 'timestamp': 0.6},
 {'path': 'videos/observation.image_episode_000000.mp4', 'timestamp': 0.7},
 {'path': 'videos/observation.image_episode_000000.mp4', 'timestamp': 0.8},
 {'path': 'videos/observation.image_episode_000000.mp4', 'timestamp': 0.9},
 {'path': 'videos/observation.image_episode_000000.mp4', 'timestamp': 1.0},
 {'path': 'videos/observation.image_episode_000000.mp4', 'timestamp': 1.1},
 {'path': 'videos/observation.image_episode_000000.mp4', 'timestamp': 1.2},
 {'path': 'v

In [9]:
info = {'codebase_version': 'v1.6',
 'fps': 10,
 'video': True,
 'encoding': {'vcodec': 'libsvtav1', 'pix_fmt': 'yuv420p', 'g': 2, 'crf': 30}}

In [10]:
root.attrs.put(info)

print(root.attrs)

In [11]:
root.attrs['a'] = 1

In [12]:
root.attrs.asdict()

{'codebase_version': 'v1.6',
 'encoding': {'crf': 30, 'g': 2, 'pix_fmt': 'yuv420p', 'vcodec': 'libsvtav1'},
 'fps': 10,
 'video': True,
 'a': 1}